In [1]:
%load_ext autoreload
%autoreload 2

# DeepSeek on VKS documents

## Using PostgreSQL to retrieval VKS documents


Define global variables

In [2]:
import os

from langchain_community.llms import VLLMOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

from modules import utils

from IPython.display import display, Markdown, Latex

In [3]:
ENV_FILE_PATH = "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/env"
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
MAX_TOKENS = 32700
COLLECTION_NAME = "my_docs"

Load environment variables

In [4]:
envs = utils.load_env_to_dict(ENV_FILE_PATH)

## Prepare PostgreSQL vectorstore

In [5]:
from langchain_postgres.vectorstores import PGVector
from langchain_huggingface.embeddings import HuggingFaceEmbeddings


In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
)

/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
vector_store = PGVector(
    embeddings=embeddings,
    collection_name=COLLECTION_NAME,
    connection=envs["POSTGRESQL_URI_2"],
    use_jsonb=True,
)

## Retrieval

In [8]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [9]:
res = vector_store.similarity_search("VKS là gì?")
utils.print_documents(res)

Amount of documents: 4
Document 0:
VKS là gì?

VKS (VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn đơn giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là một nền tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý và triển khai các ứng dụng container trên môi trường phân tán.

Những điểm nổi bật của VKS

Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS sẽ giải phóng bạn khỏi gánh nặng quản lý Control Plane của Kubernetes, giúp bạn tập trung vào việc phát triển ứng dụng.

Hỗ trợ các phiên bản Kubernetes mới nhất: VKS luôn cập nhật những phiên bản Kubernetes mới nhất (minor version từ 1.27, 1.28, 1.29) để đảm bảo bạn luôn tận dụng được những tính năng tiên tiến nhất.

Kubernetes Networking: VKS tích hợp Calico CNI, mang lại tính hiệu quả và bảo mật cao.

Upgrade seamlessly: VKS hỗ trợ nâng cấp giữa các phiên bản Kubernetes một cách dễ dàng và nhanh chóng, giúp bạn luô

## Integrated with DeepSeek R1

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [11]:
# Connect to vLLM API
llm = ChatOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=envs['VLLM_HOST_URL_2'] + "/v1/",
    model_name=MODEL_NAME,
    max_tokens=MAX_TOKENS,
    streaming=True,
)

prompt = ChatPromptTemplate.from_template("""
Bạn là một trợ lí ảo nhằm trả lời các CÂU HỎI cho khách hàng của VNGCloud.
Câu trả lời của bạn phải có liên quan đến NGỮ CẢNH được cung cấp. Nếu không liên quan hãy yêu cầu cung cấp thêm thông tin.
Bạn PHẢI trả lời bằng TIẾNG VIỆT

CÂU HỎI: {query}

NGỮ CẢNH:
{context}

TRẢ LỜI:
""")

chain = prompt | llm

## Prepare the context

In [12]:
query = "VKS là gì?"
context = utils.from_documents_to_context(vector_store.similarity_search(query))

display(Markdown(context[:1000]))

- Document 1:
VKS là gì?

VKS (VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn đơn giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là một nền tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý và triển khai các ứng dụng container trên môi trường phân tán.

Những điểm nổi bật của VKS

Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS sẽ giải phóng bạn khỏi gánh nặng quản lý Control Plane của Kubernetes, giúp bạn tập trung vào việc phát triển ứng dụng.

Hỗ trợ các phiên bản Kubernetes mới nhất: VKS luôn cập nhật những phiên bản Kubernetes mới nhất (minor version từ 1.27, 1.28, 1.29) để đảm bảo bạn luôn tận dụng được những tính năng tiên tiến nhất.

Kubernetes Networking: VKS tích hợp Calico CNI, mang lại tính hiệu quả và bảo mật cao.

Upgrade seamlessly: VKS hỗ trợ nâng cấp giữa các phiên bản Kubernetes một cách dễ dàng và nhanh chóng, giúp bạn luôn cập nhật những cải 

In [13]:
query = "VKS là gì?"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Okay, so I need to figure out what VKS is based on the information provided. Let's see, the user gave me four documents, and all of them mention VKS in the context of VNGCloud. 

Looking at Document 1, it clearly states that VKS stands for VNGCloud Kubernetes Service. It's a managed service on VNGCloud that helps simplify deploying and managing container applications. Kubernetes is an open-source platform by Google for container orchestration. This document explains the features of VKS, like automatic control plane management, support for the latest Kubernetes versions, networking with Calico, and more.

Document 2 is shorter but reiterates that VKS is VNGCloud's managed Kubernetes service. Document 3 provides updates and improvements to VKS, showing that it's actively developed and supported. Finally, Document 4 details how to use Terraform with VKS, which indicates that VKS is integrated with infrastructure-as-code tools, making it flexible for users.

Putting it all toge

In [14]:
query = "Làm sao để expose service trong VKS?"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Ồ, khách hàng đang hỏi cách expose service trong VKS. Tôi xem qua các document được cung cấp, nhưng hầu hết đều nói về việc tạo cluster và node group qua Terraform, cùng với việc tạo Service Account và activate dịch vụ VKS. Không có thông tin cụ thể nào về việc expose service.

Thông thường, để expose service trong Kubernetes, người ta sử dụng các phương pháp như tạo Service loại LoadBalancer, NodePort, hoặc sử dụng Ingress. Nhưng các document ở đây không đề cập đến cách thực hiện các步骤 này trong bối cảnh VKS của VNG Cloud.

Có thể tôi cần yêu cầu khách hàng cung cấp thêm thông tin về cách họ đang triển khai ứng dụng, hoặc liệu họ có gặp khó khăn gì khi cấu hình các service không. Hoặc, có thể họ đang tìm kiếm hướng dẫn cụ thể cho môi trường VKS, mà các document hiện tại không đề cập đến.

Tôi cũng có thể đề xuất một số phương pháp chung để expose service, nhưng nếu không có thông tin cụ thể về VKS, có thể không chính xác. Vì vậy, tốt nhất là yêu cầu thêm chi tiết từ khách 

In [15]:
query = "Hoàng Sa và Trường Sa là của ai?"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Được rồi, tôi cần xử lý câu hỏi và ngữ cảnh mà người dùng đưa ra. Câu hỏi là "Hoàng Sa và Trường Sa là của ai?", ενώ các Document 1 đến Document 4 đều liên quan đến công nghệ Kubernetes, bao gồm Label, Taint, Node Group và CSI. Không hề có thông tin nào về chủ đề địa lý hay chính trị liên quan đến quần đảo Hoàng Sa và Trường Sa.

Do đó, câu trả lời cần phải giải thích rằng các tài liệu cung cấp không liên quan đến câu hỏi và yêu cầu người dùng cung cấp thêm thông tin hoặc đặt câu hỏi khác liên quan đến Kubernetes. Tôi cũng phải giữ giọng điệu lịch sự và sẵn lòng giúp đỡ nếu có các câu hỏi khác.



GENERATED RESPONSE


Câu hỏi của bạn không liên quan đến nội dung của các Document được cung cấp. Các Document chủ yếu trình bày về Label, Taint, Node Group và CSI trong Kubernetes. Để được hỗ trợ, vui lòng cung cấp thêm thông tin liên quan đến các chủ đề này.

In [16]:
query = "Tạo cho tôi PVC 20Gb trong vks"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Được yêu cầu tạo PVC 20Gb trong VKS, tôi查看 cung cấp thông tin. Tài liệu 1提到 PVC kích cỡ tối thiểu đã giảm từ 20GB xuống 1GB, nghĩa là có thể tạo được 20GB. Tài liệu 2 và 3 cung cấp các cách tạo PVC qua YAML, bao gồm các tham số cần thiết.

Tôi cần tạo một file YAML cho PVC với dung lượng 20Gi, sử dụng StorageClass mặc định. Theo tài liệu, tôi có thể đặt storageClassName và kích thước lưu trữ. Ngoài ra, do đang sử dụng gặp các annotate isPOC: "true" nếu cần dùng ví POC.

Sau đó, tôi sẽ chạy lệnh kubectl apply để triển khai file YAML này vào cluster Kubernetes. Cuối cùng, tôi kiểm tra trạng thái PVC để đảm bảo nó đã được tạo thành công.



GENERATED RESPONSE


Để tạo một Persistent Volume Claim (PVC) với dung lượng 20GB trong VNGCloud Kubernetes Service (VKS), bạn có thể làm theo các bước sau đây:

### Tạo file YAML cho PVC

Tạo một file YAML mới, chẳng hạn là `pvc-20gb.yaml`, với nội dung sau:

```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: pvc-20gb
spe

In [17]:
query = "Làm sao để cấu hình volume type khi dùng pvc trong vks"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Được rồi, tôi cần giúp khách hàng cấu hình volume type khi dùng PVC trong VKS. Từ câu hỏi và ngữ cảnh cung cấp, tôi thấy khách hàng đang sử dụng Kubernetes và có một.yaml để tạo PVC. Họ muốn biết cách cấu hình loại volume khi tạo PVC.

Trong document 1, tôi nhìn thấy rằng PVC được cấu hình với storageClassName. Điều này cho thấy rằng storage class quyết định loại volume được sử dụng. Vì vậy, bước đầu tiên là kiểm tra storage class có sẵn.

Tiếp theo, tôi nghĩ cần giải thích cách thêm annotation vào PVC để chỉ định volume type. Trong document 3, có đề cập đến việc thêm annotation bs.csi.vngcloud.vn/volume-type với ID cụ thể. Điều này giúp hệ thống hiểu nên dùng loại volume nào.

Также, tôi cần mention việc thay đổi storage class mặc định sang SSD IOPS 3000, như được cập nhật trong document 4. Khách hàng có thể cần tạo storage class mới nếu muốn tiếp tục sử dụng loại cũ.

Cuối cùng, tôi phải hướng dẫn họ kiểm tra các volume type có sẵn và cách cập nhật PVC existing nếu cần th